In [ ]:
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC


import cv2
import numpy as np
import random as rand

import glob
import time


import joblib

from matplotlib.pyplot import imread

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

%matplotlib inline

from helpers import convert
from scipy.ndimage import label
# from featuresourcer import FeatureSourcer


from moviepy.editor import VideoFileClip
from IPython.display import HTML
import functools

from skimage.feature import hog
import numpy as np
from helpers import convert 


In [ ]:

sourcer_params = {
  'color_model': 'yuv',                # hls, hsv, yuv, ycrcb
  'bounding_box_size': 64,             #
  'number_of_orientations': 11,        # 6 - 12
  'pixels_per_cell': 16,               # 8, 16
  'cells_per_block': 2,                # 1, 2
  'do_transform_sqrt': True
}

start_frame = imread("Data/vehicles/KITTI_extracted/5364.png")

ppc_N = sourcer_params['pixels_per_cell']

In [ ]:
def    newFrame(strip):
    # sourcer.new_frame(strip)
    return 0

##this function have to return hog features
def hogSliceFn(strip, resizedX , y, boundingBoxSizeX, boundingBoxSizeY):
    features = 0
    return features
    # sourcer.features(strip)
    # return sourcer.slice( resizedX, y, w_pix = boundingBoxSizeX, h_pix = boundingBoxSizeY)

In [ ]:
# ##parameters

# vehiclesFeatures = vehicles_features 
# nonVehiclesFeatures = nonvehicles_features

# totalVehicles,totalNonVehicles = total_vehicles,total_nonvehicles



# def    newFrame(strip):
#     sourcer.new_frame(strip)
#     return 0

# ##this function have to return hog features
# def hogSliceFn(strip, resizedX , y, boundingBoxSizeX, boundingBoxSizeY):

#     # sourcer.features(strip)
#     return sourcer.slice( resizedX, y, w_pix = boundingBoxSizeX, h_pix = boundingBoxSizeY)


In [1]:

##scaling features for training 
def scalingFn(vehiclesFeatures, nonVehiclesFeatures,totalVehicles,totalNonVehicles, pathSaveModel = 'scaler.pkl'):
    ##put all features vectors beside eachothers
    unscaledX = np.vstack((vehiclesFeatures, nonVehiclesFeatures)).astype(np.float64)
    
    #In SVM, data normalization is required to use all kernels related to distance calculation.
    ##standardizion (mean = 0, std = 1) the data which by centerized it and divide by standaard division. betwise the normalize it (the data range bet 0 to 1)
    scalerM = StandardScaler() #creating the module for the standardization
    scaler = scalerM.fit(unscaledX) #Compute the mean and standard devision to be used for later scaling.
    x = scaler.transform(unscaledX) #Perform standardization by centering and scaling.
    y= np.hstack((np.ones(totalVehicles),np.zeros(totalNonVehicles)))


    print ("Saving models...")
    #save the scaler model
    joblib.dump(scaler,pathSaveModel )
    print ("Saving models...")

    return x, y, scaler


In [ ]:
print("Scaling features...")
t_start = time.time()

xScaledFeatured,yScaledFeatured,scaler = scalingFn(vehiclesFeatures, nonVehiclesFeatures,totalVehicles,totalNonVehicles, pathSaveModel = 'scaler.pkl')

print("...Done")
print("Time Taken:", np.round(time.time() - t_start, 2))
print(" x shape: ", xScaledFeatured.shape, " y shape: ", yScaledFeatured.shape)

In [ ]:
##training the data 

def trainFn(xScaledFeatured,yScaledFeatured, pathSaveModel = 'svc.pkl'):

    #split the data bet the training which 70% vs the test wich is 30%, also the chosen data are randomly by precent (rand_state)
    xTrain, xTest, yTrain, yTest = train_test_split(xScaledFeatured,yScaledFeatured, test_size = 0.3,  random_state =rand.randint(1,100))
    
    # SVM is used for both classification and regression problems.
    # Scikit-learn's method of Support Vector Classification (SVC) can be extended to solve regression problems as well. 
    # That extended method is called Support Vector Regression (SVR).
    
    #creating the model
    #svm with linear (also line spearate between the samples ) kernal not segmoid or tanh or relu, between the bars of neural nodes
    svc = LinearSVC() 

    #traning the model with the traintng data data
    svc.fit(xTrain,yTrain)

    #check the accuracy of the model by the testing data
    accuracy = svc.score(xTest,yTest)
    
    print ("Saving models...")
    joblib.dump(svc, pathSaveModel)
    print("...Done")
  
    return svc , accuracy


In [ ]:
print("Training classifier...")
t_start = time.time()

svc , accuracy = trainFn(xScaledFeatured,yScaledFeatured, pathSaveModel = 'svc.pkl')



print("...Done")
print("Time Taken:", np.round(time.time() - t_start, 2))
print("Accuracy: ", np.round(accuracy, 4))

In [ ]:
##predict func 

def predictFn(frameFeature, svc, ScalerModel):
    
    #scale (standrize) the feature, which i use to predict, 
    frameScaled = ScalerModel.transform([frameFeature])

    #make the prediction
    frameClass = svc.predict(frameScaled)

    #return the frame class as int
    return np.int(frameClass[0]) 

In [ ]:
#load the models
print ("Loading models...")

scaler = joblib.load("scaler.pkl")
svc = joblib.load("svc.pkl")

print("...Done")

In [ ]:
# sourcer = FeatureSourcer(sourcer_params, start_frame)

# f1 = sourcer.features(nonvehicle_imgs[rand.randint(0, totalNonVehicles)])
# f2 = sourcer.features(vehicle_imgs[rand.randint(0, totalVehicles)])
# f3 = sourcer.features(vehicle_imgs[rand.randint(0, totalVehicles)])
# f4 = sourcer.features(nonvehicle_imgs[rand.randint(0, totalNonVehicles)])


# print(predictFn(f4, svc, scaler))


In [ ]:
##slider : we choose only part of the frame which we call strip to extract the hog feature from it,
#   then predic it every slice on it

#prepare the part we work on to extract the feature, and classify each half cel on it
def prepareSlider(frame , yStart,windowSize,boundingBoxSize):
    
    scaler = windowSize / boundingBoxSize ##for ex 80 /64
    
    #for the strip we get the end
    yEnd = yStart + windowSize

    #the width of the strip which we use it to resize the the stip
    #which will be smaller than the fram width frame by (boundaryBox(64) / windowSize(80) ) 
    #and equal to mulible of boundary boxes by (widthFrame(1024) / windowSize(80)) ~ 12.55
    newWidth = np.int(frame.shape[1] / scaler) # width = boundaryBox(64) * (widthFrame / windowSize(80))

    #take the required strip from the frame
    strip = frame[yStart : yEnd, :]

    #resize this strip
    strip = cv2.resize(strip,(newWidth,boundingBoxSize ))

    return strip, scaler


#to locate the vehicles from the frame
def locateVehicle(frame, yWindowPosition, windowSize, boundingBoxSize, inc, svcModelPath = 'svc.pkl',ScalarModelPath = 'scaler.pkl') :
    
    #load the classifier model
    svc =  joblib.load(svcModelPath)
    ScalarModel = joblib.load(ScalarModelPath)

    #prepare the part we work on to extract the feature, and classify each half cel on it
    strip , scaler = prepareSlider(frame , yWindowPosition,windowSize,boundingBoxSize)

    #boxes which we found the car on it, and want to draw pox on it
    boxes = []

    #make hog to every channel of the strip , in the new frame
    newFrame(strip)

    #just count number of the boxes (blocks) in the width of the strip and substract one from it
    #by taking the floor (stripWidth / boundingBoxSize) => 12 to there no be fractions, then multyply it on  boundingBoxSize => 12 * 64
    # the substract one boundingBoxSize from it => 11 * 64
    #which will be the last x we extract the hog and classfy it 
    xEnd = (strip.shape[1] // boundingBoxSize - 1) * boundingBoxSize

    for resizedX in range(0, xEnd, inc): #if inc = 8 #0,8,16,24, 32, 40, 56, 64, 72, ......

        #extract the features from one slice from the the strip
        # if resizedX = 16 , then iside xStart = floor( 16/pixelpercell) = 1 , width = floor(boundingBoxSize / pexelPerCell) =64/16 = 4, xend = 1 + 4 -1  = 4
        #yStart = 0 / pixelPerCell = 0, height =boundingBoxSize / pexelPerCell  = 4 , yend = 0 + 4 
        #get the feature vector which compines from the 3 channels put besides each other
            #yStartof the real frame  which is the same of the y of window box(yWindowPosition) 
            #because we don't move from the start of the strip = 0 which is represent the yWindowPosition #so the inY here is  = 0
            ### we go through only the top of the strip because if that slice is a part of vehicle, so the whole window is a vehicle 
            #####also maybe if we get part of the car, it will be a car for all the window postion
        features = hogSliceFn(strip, resizedX , 0, boundingBoxSize  , boundingBoxSize)


        #after extraction 
        #we classify (predict)
        #if it's a car we get the xStart position of it in the real fram image not the strip
        if predictFn(features, svc, ScalarModel):

            #xStart = resizedX of the strip* (windowSize / boundingBoxSize) ##for ex 16 *( 80 /64)
            #because we divide the width of the frame by the scaler, so we multyply it again
            xStartV = np.int(scaler * resizedX) 

            #we append every thing about that car, like xStart, yStart which is the same of the y of window box 
            #because we don't move from the start of the strip = 0 which is represent the yWindowPosition
            #also maybe if we get part of the car, it will be a car for all the window postion
            boxes.append((xStartV, yWindowPosition, windowSize))


    return boxes, strip



In [ ]:

#get all copies of the images 
def slider(frame,  windowSizes, stripPostions , boundingBoxSize, inc, svcModelPath = 'svc.pkl' , ScalarModelPath = 'scaler.pkl'):

    boxedImages = []
    strips = []
    boungingBoxesTotal = [] 
    ##we go through differnt strip postions on the original frame 410, 390, 380, 380
    # to see if there is a vehicles close to the my car or far away
    #also we got through differen windows because
    #maybe there different size of the vichels 

    ######## the main reason ###################
    # because we classify only the top of the stip 
    # so we can deal if it's a shadow (false positive) or a car (true positive) so we classify the same vehicle from diffiernt postion
    for yWindowPosition, windowSize in zip(stripPostions, windowSizes):

        # Get the vehicles boxes (xStart, yStart, windowsSize) of the original frame from the first strip , second, third, and forth
        boundingBoxes, strip = locateVehicle(frame, yWindowPosition, windowSize, boundingBoxSize, inc, svcModelPath,ScalarModelPath)

        for boundingBox in boundingBoxes:#################################added#########################
            boungingBoxesTotal.append(boundingBox)

        #draw rectangles on each vehicle (of that strip), on a copy from the same original frame, no change for the frame
        boxedImage = drawBoxes(frame, boundingBoxes)

        #collect these copies of the images for the same original frame
        boxedImages.append(boxedImage)

        #collect the strips we work on together
        strips.append(strip)


    return boungingBoxesTotal, boxedImages, strips
    



In [ ]:
frame1 = imread("Project_data/test_images/test1.jpg")
frame2 = imread("Project_data/test_images/test2.jpg")
frame3 = imread("Project_data/test_images/test3.jpg")
frame4 = imread("Project_data/test_images/test4.jpg")
frame5 = imread("Project_data/test_images/test5.jpg")
frame6 = imread("Project_data/test_images/test6.jpg")
windowSizes = 80, 120, 150, 180
stripPostions = 410, 390,  380, 380

boundingBoxSize = sourcer_params['bounding_box_size']
# slider1 = Slider(sourcer = sourcer, ScalerModelPath="scaler.pkl", svcModelPath="svc.pkl", increment = 8)


In [ ]:

#test the sliding windows
boungingBoxesTotal, boxedImages, strips = slider(frame5 , windowSizes, stripPostions , boundingBoxSize, 8)



In [ ]:
# showImages(strips, ncols=  len(strips), nrows=  1, width=  15, height=  3)
# showImages(boxedImages, ncols = len(boxedImages), nrows = 1, width = 15, height = 3)